## Problem 1

Find the absolute minimum of the function

\begin{equation}
    f(x) = f(x_1, x_2) = x_1 e^{-{x_1}^2 -{x_2}^2}
\end{equation}

in the domain $x \in \mathbb{R}^2$ (unconstrained problem).

The initial point is $x^0 = (-0.6, -0.3)$.

In [67]:
import numpy as np
import pandas as pd

# Define the objective function f(x) that you want to minimize
def f(x):
    x1, x2 = x
    return x1 * np.exp(-x1**2 - x2**2)

# Define the Jacobian matrix (partial derivatives of f with respect to x1 and x2)
def grad_f(x):
    x1, x2 = x
    df_dx1 = np.exp(-x1**2 - x2**2) - 2 * x1**2 * np.exp(-x1**2 - x2**2)
    df_dx2 = -2 * x1 * x2 * np.exp(-x1**2 - x2**2)
    return np.array([df_dx1, df_dx2])

# Levenberg-Marquardt optimization with iteration data
def levenberg_marquardt(x0, tol, J, beta_init):
    x = np.array(x0)
    β = beta_init
    optimization_data = []
    k = 0

    while(True):
        # Compute the gradient and Hessian (approximately the Jacobian)
        g = -grad_f(x)
        hessian = np.outer(g, g)
        np.fill_diagonal(hessian, hessian.diagonal() + β)

        # Compute the search direction using the pseudo-inverse
        s = np.linalg.pinv(hessian).dot(g)

        # Calculate the
        fx = f(x)

        # Update the parameter vector (λ = 1)
        x_new = x + s

        # Append data for this iteration
        optimization_data.append({'Iteration': k, 'x1': x[0], 'x2': x[1], 'f(x)': fx, '|f(x_next) - f(x)|': f(x_new) - fx, 'β': β})

        # Check for convergence based on gradient norm
        if abs(f(x_new) - fx) < tol:
            break

        if k >= J:
            break

        # Update beta based on convergence
        if f(x_new) < fx:
            β /= 10
            x = x_new
        else:
            β *= 10
        k += 1

    return x, optimization_data

# Initial guess
initial_guess = [-0.6, -0.3]

# Set the tolerance for gradient norm
tolerance = 1e-15

# Perform LM optimization
optimal_x, optimization_data = levenberg_marquardt(initial_guess, tol=tolerance, J=100, beta_init=0.01)

# Create a Pandas DataFrame from the optimization data
df = pd.DataFrame(optimization_data)

# Print the results
print("Optimized x1:", optimal_x[0])
print("Optimized x2:", optimal_x[1])
print("Optimized f(x):", f(optimal_x))
print("\nOptimization Data:")
print(df)


Optimized x1: -0.7071068193998944
Optimized x2: -2.980431581506263e-37
Optimized f(x): -0.42888194248035216

Optimization Data:
    Iteration        x1            x2      f(x)  |f(x_next) - f(x)|     β
0           0 -0.600000 -3.000000e-01 -0.382577        3.825217e-01  0.01
1           1 -0.600000 -3.000000e-01 -0.382577        3.274279e-01  0.10
2           2 -0.600000 -3.000000e-01 -0.382577       -4.023621e-02  1.00
3           3 -0.764615 -8.835218e-02 -0.422813        4.619382e-01  0.10
4           4 -0.764615 -8.835218e-02 -0.422813       -4.923016e-03  1.00
..        ...       ...           ...       ...                 ...   ...
84         84 -0.707107 -1.473192e-35 -0.428882       -2.275957e-15  1.00
85         85 -0.707107 -2.095411e-36 -0.428882        6.360468e-13  0.10
86         86 -0.707107 -2.095411e-36 -0.428882       -1.221245e-15  1.00
87         87 -0.707107 -2.980432e-37 -0.428882        3.256839e-13  0.10
88         88 -0.707107 -2.980432e-37 -0.428882       -5.5